# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [11]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [12]:
#Load the weather dataframe from WeatherPy and clean the dataframe to drop NaN values
weather_data_to_load = "../WeatherPy/output_data/cities.csv"
weather_data = pd.read_csv(weather_data_to_load)
del weather_data["Unnamed: 0"]
weather_data = weather_data.dropna()
weather_data


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,albany,42.60,-73.97,39.99,68,82,3.00,US,1609200607
1,faanui,-16.48,-151.75,80.92,69,10,7.61,PF,1609200836
2,odessa,46.48,30.73,46.40,100,90,15.66,UA,1609200836
3,college,64.86,-147.80,12.20,78,75,3.36,US,1609200837
4,busselton,-33.65,115.33,72.00,52,80,3.00,AU,1609200837
...,...,...,...,...,...,...,...,...,...
579,correntina,-13.34,-44.64,71.65,64,42,3.22,BR,1609200891
580,miri,4.41,114.01,77.00,94,40,5.82,MY,1609200891
581,bajil,15.06,43.29,73.18,74,0,1.66,YE,1609200891
582,orlik,52.52,99.83,-34.11,66,0,1.74,RU,1609200891


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [13]:
#Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [14]:
#Use the Lat and Lng as locations and Humidity as the weight
locations = weather_data[["Lat", "Lng"]].astype(float)
humidity = weather_data["Humidity"].astype(float)

In [19]:
#Create the map
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

#Add the heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [27]:
#Narrow down the cities such that:
#A max temperature lower than 80 degrees but higher than 70
best_temperature = (weather_data["Max Temp"] < 80) & (weather_data["Max Temp"] > 70)

#Wind speed less than 10 mph.
best_wind_speed = (weather_data["Wind Speed"] < 10)
                   
#Zero cloudiness
zero_cloudiness = (weather_data["Cloudiness"] == 0)

#Create a variable to hold all the conditions
all_conditions = best_temperature & best_wind_speed & zero_cloudiness

#Use a boolean mask to filter the dataframe to fit the weather conditons
perfect_weather_df = weather_data[all_conditions]
perfect_weather_df = perfect_weather_df.dropna()
perfect_weather_df = perfect_weather_df.reset_index()
perfect_weather_df.head(10)

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,16,lazaro cardenas,17.96,-102.20,75.79,69,0,6.02,MX,1609200750
1,28,konongo,6.62,-1.22,71.51,75,0,1.34,GH,1609200839
2,69,nong phai,15.99,101.06,77.00,69,0,3.18,TH,1609200843
3,81,comodoro rivadavia,-45.87,-67.50,77.00,44,0,8.05,AR,1609200844
4,216,ler,8.30,30.14,76.50,22,0,5.82,SS,1609200858
5,258,tabou,4.42,-7.35,78.55,85,0,6.33,CI,1609200862
6,317,caravelas,-17.71,-39.25,74.61,86,0,5.77,BR,1609200867
7,337,riachao das neves,-11.75,-44.91,73.51,57,0,2.21,BR,1609200735
8,351,korhogo,9.45,-5.63,72.91,22,0,4.47,CI,1609200871
9,367,mahajanga,-15.72,46.32,77.00,94,0,2.24,MG,1609200874


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [33]:
#Store the dataframe above to new variable
hotel_df = perfect_weather_df.loc[[0,1,2,3,4,5,6,7,8,9]]

#Add a "Hotel Name" column to the Dataframe
hotel_df["Hotel Name"] = ""

hotel_df

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,16,lazaro cardenas,17.96,-102.20,75.79,69,0,6.02,MX,1609200750,
1,28,konongo,6.62,-1.22,71.51,75,0,1.34,GH,1609200839,
2,69,nong phai,15.99,101.06,77.00,69,0,3.18,TH,1609200843,
3,81,comodoro rivadavia,-45.87,-67.50,77.00,44,0,8.05,AR,1609200844,
4,216,ler,8.30,30.14,76.50,22,0,5.82,SS,1609200858,
5,258,tabou,4.42,-7.35,78.55,85,0,6.33,CI,1609200862,
6,317,caravelas,-17.71,-39.25,74.61,86,0,5.77,BR,1609200867,
7,337,riachao das neves,-11.75,-44.91,73.51,57,0,2.21,BR,1609200735,
8,351,korhogo,9.45,-5.63,72.91,22,0,4.47,CI,1609200871,
9,367,mahajanga,-15.72,46.32,77.00,94,0,2.24,MG,1609200874,


In [38]:
#Create a dictionary holding the parameters we want the request to entail
params = {"radius": 5000,
         "types": "lodging",
         "keyword": "hotel",
         "key": g_key}

#Use the lat/lng we recovered to identify hotels
for index,row in hotel_df.iterrows():
    #Get lat, lng from dataframe
    lat = row["Lat"]
    lng = row["Lng"]

    #Change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    #Make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except:
        print("Nearest hotel not found... skipping.")
        
hotel_df

Nearest hotel not found... skipping.


,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,16,lazaro cardenas,17.96,-102.20,75.79,69,0,6.02,MX,1609200750,Hotel de Casablanca
1,28,konongo,6.62,-1.22,71.51,75,0,1.34,GH,1609200839,Telecentre Hotel
2,69,nong phai,15.99,101.06,77.00,69,0,3.18,TH,1609200843,Green Vale Organic Farm
3,81,comodoro rivadavia,-45.87,-67.50,77.00,44,0,8.05,AR,1609200844,Lucania Palazzo Hotel
4,216,ler,8.30,30.14,76.50,22,0,5.82,SS,1609200858,
5,258,tabou,4.42,-7.35,78.55,85,0,6.33,CI,1609200862,hotêl doufoulougou
6,317,caravelas,-17.71,-39.25,74.61,86,0,5.77,BR,1609200867,Pousada Liberdade
7,337,riachao das neves,-11.75,-44.91,73.51,57,0,2.21,BR,1609200735,Hotel Cristo Rei
8,351,korhogo,9.45,-5.63,72.91,22,0,4.47,CI,1609200871,Hotel Non Stop
9,367,mahajanga,-15.72,46.32,77.00,94,0,2.24,MG,1609200874,Baobab Tree Hotel & Spa


In [40]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [42]:
# Add marker layer ontop of heat map
figure_layout = {
    'width': '900px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(layout=figure_layout,zoom_level=2,center=(15,25))

#Create the hotel symbol layer
hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

#Add both layers 
fig.add_layer(heat_layer)
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…